In [378]:
from transformers import BertTokenizerFast, AutoTokenizer, BertModel
import srsly
from tqdm import tqdm
from datasets import Dataset, DatasetDict, load_from_disk
import spacy

In [355]:
tokenizer = BertTokenizer.from_pretrained('tokenizer')

In [356]:
tokenizer.vocab['[BLANK]']

1

In [357]:
tokenizer.encode(['a', '[BLANK]'])

[101, 143, 1, 102]

In [349]:
train_data = list(srsly.read_jsonl('cluener/train.json'))

In [297]:
val_data = list(srsly.read_jsonl('cluener/dev.json'))

In [406]:
def process(data_ls):
    dataset = {'text':[], 'spans':[], }
    for data in tqdm(data_ls):
        dataset['text'].append(data['text'])
        entities = []
        # mapping = tokenizer(data['text'], add_special_tokens=False, return_offsets_mapping=True)['offset_mapping']
        # tokens = tokenizer.tokenize(data['text'], add_special_tokens=False)
        # dataset['tokens'].append(tokens)
        # spans = []
        for label in data['label']:
            for ent in data['label'][label]:
                for span in data['label'][label][ent]:
                    char_start = span[0]
                    char_end = span[1]
                    entity = []
                # # try:
                # for i, v in enumerate(mapping):
                #     if v[0] == char_start and v[1] == char_end:
                #         entity.append(i)
                #         entity.append(i+1)
                #     if v[0] == char_start and v[1]!= char_end:
                #         entity.append(i)
                #     elif v[1] == char_end and v[0] != char_start:
                #         entity.append(i)
                    entity.append(char_start)
                    entity.append(char_end)
                    entity.append(label)
                    entity.append(ent)
                # print(entity)
                    entity = list(map(str, entity))
                    assert len(entity) == 4, (data, tokens)
                # except AssertionError:
                #     print(entity)
                    entities.append(entity)
        dataset['spans'].append(entities)
    return dataset

In [407]:
train_dict = process(train_data)

100%|██████████| 10748/10748 [00:00<00:00, 161794.73it/s]


In [408]:
val_dict = process(val_data)

100%|██████████| 1343/1343 [00:00<00:00, 87983.23it/s]


In [409]:
trainset = Dataset.from_dict(train_dict)

In [410]:
valset = Dataset.from_dict(val_dict)

In [411]:
print(trainset[13])

{'text': '会议批准了中国与欧盟海军、多国海上力量和北约等就在“国际推荐通行走廊”', 'spans': [['5', '11', 'government', '中国与欧盟海军'], ['20', '21', 'government', '北约']]}


In [412]:
ds = DatasetDict()
ds['train'] = trainset
ds['validation'] = valset

In [414]:
ds.save_to_disk('clue_ner_charspan')

In [415]:
ds = load_from_disk('clue_ner_charspan')

In [416]:
ds['train'][17]

{'text': '她写道：抗战胜利时我从重庆坐民联轮到南京，去中山陵瞻仰，也到秦淮河去过。然后就去北京了。',
 'spans': [['11', '12', 'address', '重庆'],
  ['18', '19', 'address', '南京'],
  ['40', '41', 'address', '北京'],
  ['22', '24', 'scene', '中山陵'],
  ['30', '32', 'scene', '秦淮河']]}